<a href="https://colab.research.google.com/github/richarddascombe/CodeWars/blob/main/CodeWars_theLift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Codewars The Lift
https://www.codewars.com/kata/58905bfa1decb981da00009e/train/python

In [6]:
class Dinglemouse(object):
    """
    A class to simulate the operation of a lift (elevator).
    """
    # Create the initialiser (basically the blueprint for creating a lift object)
    # we are having queues as a class attribute (rather than an instance method attribute) as it is a starting condition and we are looking to solve this single starting condition
    def __init__(self, queues, capacity):
        """
        Initializes the lift simulation.

        Args:
            queues (list of lists): A list where each inner list represents a floor's queue.
                                    Each number in an inner list is the destination floor
                                    of a person waiting on that floor.
                                    Example: [[], [], [5, 1], [], []] means 2 people on floor 2
                                    want to go to floor 5 and floor 1 respectively.
            capacity (int): The maximum number of people the lift can hold.
        """
        self.queues = [list(q) for q in queues]  # Make a deep copy to avoid modifying original input (not .copy() wouldn't cut the mustard!)
        self.capacity = capacity
        self.current_floor = 0  # Lift starts at the ground floor (floor 0)
        self.passengers = []    # List of destination floors for people currently in the lift
        self.visited_floors = [0] # A list Recording floors visited, starting at 0
        self.direction = 1      # 1 for up, -1 for down. Initial direction is up.
        self.max_floor = len(queues) - 1  # Determine the INITIAL highest floor based on queues N.B. highest floor is effectively the highest non-empty queue element
        #self.max_floor = max(element for sublist in queues for element in sublist) # Determine the highest floor based larges value in queues

        # A flag to help determine if the lift has completed a cycle without activity
        self.idle_count = 0

    def theLift(self):
        """
        This instance method simulates the lift's operation until all people are picked up
        and dropped off.

        Returns:
            list: A list of unique floors visited by the lift in order.
        """
        # The core simulation loop will go here.
        # It needs to continue as long as there are people in queues OR
        # people in the lift.

        print('max floor=',self.max_floor,'capacity=',self.capacity)

        while True:
            # 0. While the queue is not empty, there are passengers in the lift and the lift is not at floor 0
            # 1. Check for people to drop off at current_floor
            # 2. Decide if direction needs to change
            # 2. Check for people to pick up at current_floor (if space and direction matches)
            # 4. Move to the next floor

            # the lift starts at floor 0 and people at floor 0 can board immediately

            print('Welcome to floor', self.current_floor, ' i have',len(self.passengers),'passengers',self.passengers)
            print('queue is', self.queues)
            self.lift_stopping = False  # default to lift not stopping which will change if there's a drop-off or pick-up
            if self.direction==1:
                print('Going up')
            else:
                print('Going down')

            # check for passengers to drop off at this floor
            if self.current_floor in self.passengers:
                num_passengers_disembarking=self.passengers.count(self.current_floor)
                for _ in range(num_passengers_disembarking):
                    self.lift_stopping = True
                    print(f"Passenger at floor {self.current_floor} dropped off")
                    self.passengers.remove(self.current_floor)

            print('reviewing direction')
            # decide which direction to go in but don't go there yet
            if self.direction == 1:
                # if going up then continue going up if there is anyone in the queue above current_floor or if any passenger is > current_floor
                if any(self.queues[i] for i in range(self.current_floor + 1, self.max_floor + 1)) or any(passenger > self.current_floor for passenger in self.passengers):
                    self.direction = 1
                else:
                    self.direction = -1 # Change direction
            elif self.direction == -1:
                # if going down then continue going down if there is anyone in the queue below current_floor or if any passenger is < current_floor
                if any(self.queues[i] for i in range(self.current_floor)) or any(passenger < self.current_floor for passenger in self.passengers):
                    self.direction = -1
                else:
                    self.direction = 1 # Change direction

            # check for passengers to pick up at this floor
            passengers_waiting=self.queues[self.current_floor].copy()
            if len(passengers_waiting)!=0:     # N.B. any() doesn't work for queue elements of 0 ... any([0,0]) is False!!!!
                # the Lift will stop at a floor even if it is full
                self.lift_stopping = True
            # loop through passengers in queue order
            for passenger in passengers_waiting:
                # if lift not full
                if len(self.passengers) < self.capacity:
                    # if going up then only pick up passengers going up (passenger > self.current_floor)
                    if (self.direction == 1 and passenger > self.current_floor) or (self.direction == -1 and passenger < self.current_floor):
                        self.passengers.append(passenger)
                        self.queues[self.current_floor].remove(passenger)
                        print(f"Passenger at floor {self.current_floor} picked up")
                else:
                    ########!!!!!!!!
                    ######### DOES THE LIFT STOP IF THE QUEUE ONLY HAS PEOPLE GOING IN THE OPPOSITE DIRECTION
                    print('Lift is full')
                    break

            # record the visit to this floor if it has dropped off OR IF THERE WERE PASSENGERS WAITING AT FLOOR
            if self.lift_stopping:
                print('recorded stopping at floor',self.current_floor)
                self.visited_floors.append(self.current_floor)

            # check if any passengers are waiting on any floor
            if not any(self.queues):
                print('All queues are empty')
                if not self.passengers:
                    print('All passengers have dropped off')
                    self.direction=-1
                    self.idle_count += 1
                    print('idle count',self.idle_count)
                    if self.current_floor==0:
                        # no queue, no passengers, at floor 0 so done!
                        break

            if self.idle_count > 3: # avoid infinite loops
                break

            print('now moving in direction', self.direction,'current floor=',self.current_floor)
            # now we can go to the next floor
            if self.direction == 1 and self.current_floor < self.max_floor:
                self.current_floor += 1

            if self.direction == -1 and self.current_floor > 0:
                self.current_floor -= 1



        return self.visited_floors

# --- Example Usage ---
# Let's say we have 3 floors (0, 1, 2)
# Floor 0: No one waiting
# Floor 1: Person wants to go to floor 2
# Floor 2: Person wants to go to floor 0
print('----------------------- Example 1 -----------------------')
queues_example = [[], [2], [0],[0]]
print(queues_example)
lift_capacity = 1

my_lift = Dinglemouse(queues_example, lift_capacity)
journey = my_lift.theLift()
print(f"Floors visited: {journey}")
print('----------------------- Example 2 -----------------------')
# Another example
# Floor 1: 2 x people want to go to floor 2
# Floor 2: 2 x people want to go to floor 0
queues_example_2 = [[], [2, 2], [0, 0], [4], [1]] # 5 floors
print(queues_example_2)
lift_capacity_2 = 2
my_lift_2 = Dinglemouse(queues_example_2, lift_capacity_2)
journey_2 = my_lift_2.theLift()
print(f"Floors visited (example 2): {journey_2}")

----------------------- Example 1 -----------------------
[[], [2], [0], [0]]
max floor= 3 capacity= 1
Welcome to floor 0  i have 0 passengers []
queue is [[], [2], [0], [0]]
Going up
reviewing direction
now moving in direction 1 current floor= 0
Welcome to floor 1  i have 0 passengers []
queue is [[], [2], [0], [0]]
Going up
reviewing direction
Passenger at floor 1 picked up
recorded stopping at floor 1
now moving in direction 1 current floor= 1
Welcome to floor 2  i have 1 passengers [2]
queue is [[], [], [0], [0]]
Going up
Passenger at floor 2 dropped off
reviewing direction
recorded stopping at floor 2
now moving in direction 1 current floor= 2
Welcome to floor 3  i have 0 passengers []
queue is [[], [], [0], [0]]
Going up
reviewing direction
Passenger at floor 3 picked up
recorded stopping at floor 3
now moving in direction -1 current floor= 3
Welcome to floor 2  i have 1 passengers [0]
queue is [[], [], [0], []]
Going down
reviewing direction
Lift is full
recorded stopping at flo

# New section